# Lid driven cavity problem

BC are:
- top -> shifting wall: BC = 0, BC' = (K, 0)
- left, right, bot -> static wall with drag: BC = 0, BC' = 0
  


In [ ]:
from __future__ import annotations

from dataclasses import dataclass


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from ipywidgets import *
from IPython.display import display
import napari
from tqdm import tqdm
from pathlib import Path

from decimal import Decimal
import json

def to_color(u):
    x = u[..., 0]
    y = u[..., 1]
    return 


class CFDResult:
    def __init__(self, all_u: np.array, simu: CFDSimulation):
        self.all_u = all_u
        self.simu = simu
        self.all_norms = np.linalg.norm(all_u, axis=3)

    def plot_velocities(self, constant_scale=False):
        u = np.swapaxes(self.all_u, 1, 2)  # invert axe for napari
        u = np.pad(u, ((0, 0), (0, 0), (0, 0), (0, 1))) # pad last dim to be 3 so napari thinks its rgb
        viewer = napari.imshow(u)
            
    def plot_divergences(self):
        return
        def update(step = (0, self.all_u.shape[0] - 1, 1)):
            div = simu.compute_divergence(self.all_u[step])
            plt.imshow(div)
            plt.colorbar()
            plt.show()
        return interact(update)

    def energy_over_time(self, auto_plot=True, **kwargs):
        
        # We use a proxy formula for energy defined as sum of squared norm of velocities
        # all_u of shape [S, N, M, 2]
        energy = (self.all_u ** 2).sum(axis=(1, 2, 3))
        plt.plot(list(range(len(energy))), energy, label=f're={self.simu.re}, N={self.simu.N}, dt={self.simu.dt}')
        plt.title("Proxy energy over time")
        plt.legend()
        if auto_plot:
            plt.show()

    # Meaningless without context
    def re_over_time(self):
        fig, ax = plt.subplots()
        # all_u of shape (S, N, M, 2)
        re_t = np.sqrt(np.mean((self.all_u ** 2).sum(axis=(3)) ** 2, axis=(1, 2))) * self.simu.grid_size[0] / self.simu.nu
        ax.plot(list(range(len(re_t))), re_t)
        ax.set_title("Re over time")
        plt.show()

    def save(self):
        save_folder = 'runs/'
        save_name = self.simu.name + '.npy'
        with open(save_folder + save_name, 'wb') as f:
            np.save(f, self.all_u)
            self.simu.save()

    @classmethod
    def load(cls, simu):
        save_name = 'runs/' + simu.name + '.npy'
        result = cls(np.load(save_name), simu)
        return result

    def profile_at_time(self, snap, auto_plot=True, add_ref_data=True):
        re_lb = str(self.simu.re)
        
        half_x = self.simu.M // 2 - 1
        uU = self.all_u[snap, half_x, :, 0] / self.simu.K
        yL = np.array(list(range(self.simu.M))[::-1]) * self.simu.dy / 1.0
        plt.plot(yL, uU, label=f'Our: re={re_lb}, N={self.simu.N}, dt={self.simu.dt}')

        if add_ref_data:
            df = pd.read_csv('ghia_ref_u.csv', sep='\\s+')
            if re_lb in df.columns:
                plt.scatter(df['y'], df[re_lb], label='Ghia & al')
            else:
                print(f'No reference data at RE={self.simu.re}') 
                
        plt.ylabel('u/u')
        plt.xlabel('y/L')
        plt.title(f'U profile at x/L = 0.5')
        plt.legend()
        if auto_plot:
            plt.show()
        

In [ ]:
INT = np.s_[1:-1, 1:-1, :]
XINT = np.s_[2:, 1:-1, :]
XMINT = np.s_[:-2, 1:-1, :]
YINT = np.s_[1:-1, 2:, :]
YMINT = np.s_[1:-1, :-2, :]

class CFDSimulation:
    def __init__(self, re=200, steps=20, dt=1e-4, dsave=1, grid_res=32, desc_refresh=1000):
        self.re = re
        self.steps = steps
        self.dt = dt
        self.dsave = dsave
        self.grid_size = (1, 1)
        self.N = grid_res
        self.M = grid_res
        self.dx = self.grid_size[0] / self.N
        self.dy = self.grid_size[1] / self.M
        self.dim = (self.N, self.M, 2)
        self.initial_u = np.random.sample(size=self.dim)
        self.K = 1  # U, lid velocity
        
        self.nu = self.K * self.grid_size[0] / self.re
        print(f'Using nu={self.nu} for target re:{self.re}')
        self.rho = 1
        self.p = np.zeros((self.N, self.M))
        self.curr_u = np.zeros_like(self.initial_u)
        self.saved_u = np.zeros(((self.steps // self.dsave) - 1, self.N, self.M, 2))
        self.save_idx = 0
        
        self.description_refresh = desc_refresh

    def to_dict(self):
        return {
            're': self.re,
            'steps': self.steps,
            'dt': self.dt,
            'dsave': self.dsave,
            'K': self.K,
            'N': self.N,
            'M': self.M
        }
        
    @classmethod
    def from_dict(cls, d):
        obj = cls(
            re=d.get('re', 200),
            steps=d.get('steps', 20),
            dt=d.get('dt', 1e-4),
            dsave=d.get('dsave', 1)
        )
        obj.K = d.get('K', 1)
        obj.N = d.get('N', 32)
        obj.M = d.get('M', 32)
        return obj

    @property
    def name(self):
        return f'RE{self.re}_dt{self.dt}_{self.N}x{self.M}_K{self.K}_{self.steps}:{self.dsave}steps'

    def save(self):
        with open(f"runs/{self.name}.json", "w") as f:
            json.dump(self.to_dict(), f, indent=4)

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            data = json.load(f)
            return cls.from_dict(data)

    def save_curr(self):
        
        self.saved_u[self.save_idx] = self.curr_u
        self.save_idx += 1
        # print(f'saving {self.save_idx}')
        
    
    def compute_convection(s, u):
        ux = u[..., 0]
        uy = u[..., 1]
    
        dudx = (ux[2:,1:-1] - ux[:-2,1:-1]) / (2*s.dx)
        dudy = (ux[1:-1,2:] - ux[1:-1,:-2]) / (2*s.dy)
    
        dvdx = (uy[2:,1:-1] - uy[:-2,1:-1]) / (2*s.dx)
        dvdy = (uy[1:-1,2:] - uy[1:-1,:-2]) / (2*s.dy)
        conv_u = ux[1:-1,1:-1]*dudx + uy[1:-1,1:-1]*dudy
        conv_v = ux[1:-1,1:-1]*dvdx + uy[1:-1,1:-1]*dvdy
    
        conv = np.zeros_like(u)
        conv[1:-1,1:-1,0] = conv_u
        conv[1:-1,1:-1,1] = conv_v

        #for c in [0, 1]:
        #   dcdx = (u[2:,1:-1,c] - u[:-2,1:-1,c]) / (2*self.dx)
        #   dcdy = (u[1:-1,2:,c] - u[1:-1,:-2,c]) / (2*self.dy)
        #   conv[1:-1,1:-1,c] = u[1:-1,1:-1,0]*dcdx + u[1:-1,1:-1,1]*dudy
        
        return conv
    
    def compute_viscous_drag(s, u):
        lap = np.zeros_like(u)
        """
        ux = [..., 0]
        uy = [..., 1]
        lap[INT, 0] = (
            ux[XINT] - ux[INT] + ux|XMINT] / s.dx**2
          + ux[YINT] - ux[INT] + ux|YMINT] / s.dy**2
        )
        lap[INT, 1] = (
            uy[XINT] - uy[INT] + uy|XMINT] / s.dx**2
          + uy[YINT] - uy[INT] + uy|YMINT] / s.dy**2
        )
        """
        for c in [0, 1]:
            lap[1:-1,1:-1,c] = (
                (u[2:,1:-1,c] - 2*u[1:-1,1:-1,c] + u[:-2,1:-1,c]) / s.dx**2
              + (u[1:-1,2:,c] - 2*u[1:-1,1:-1,c] + u[1:-1,:-2,c]) / s.dy**2
            )
    
        return s.nu * lap
        
    def compute_updated_pressure(s, u_star, iters=50):
        p = s.p

        def update_p_ghost():
            p[1:, 0] = p[1:, 1]
            p[1:, -1] = p[1:, -2]
            p[0, 1:] = p[1, 1:]
            p[-1, 1:] = p[-2, 1:]
            p[0,0] = (p[0,1] + p[1,0])/2
            p[0,-1] = (p[0,-2] + p[1,-1])/2
            p[-1,0] = (p[-2,0] + p[-1,1])/2
            p[-1,-1] = (p[-2,-1] + p[-1,-2])/2
            
        update_p_ghost()
        
        
        rhs = np.zeros_like(p)
        
        rhs[1:-1,1:-1] = (
            (u_star[2:,1:-1,0] - u_star[:-2,1:-1,0]) / (2*s.dx)
          + (u_star[1:-1,2:,1] - u_star[1:-1,:-2,1]) / (2*s.dy)
        )
        
        #print('rhs', rhs.min(), rhs.mean(), rhs.max(), s.dt)
        
        pn = p.copy()
        for i in range(iters):
            #if i == 0:
            #   print(i, pn.min(), pn.mean(), pn.max())
            pn[1:-1,1:-1] = (
                (p[2:,1:-1] + p[:-2,1:-1]) * s.dy**2
              + (p[1:-1,2:] + p[1:-1,:-2]) * s.dx**2
              - rhs[1:-1,1:-1] * s.dx**2 * s.dy**2
            ) / (2*(s.dx**2 + s.dy**2))
            p[:] = pn[:]
        #print('END')
    
        return p


    def project_velocity(s, u_star):
        p = s.p
        u = u_star.copy()
        u[1:-1,1:-1,0] -= (p[2:,1:-1] - p[:-2,1:-1]) / (2*s.dx)
        u[1:-1,1:-1,1] -= (p[1:-1,2:] - p[1:-1,:-2]) / (2*s.dy)
        return u

    def compute_divergence(self, u):
        divergence = np.zeros_like(u)
        divergence[INT] = (u[XINT] - u[XMINT]) / 2 + (u[YINT] - u[YMINT]) / 2
        return divergence

    def setup_bc(self, u):
        # (sorry for no notation help)
        # Ut = 2Uw-Ub
        # top - sliding wall
        u[1:-1, 0, 0] = 2 * self.K - u[1:-1, 1, 0]
        #u[1:-1, 0, 0] = - u[1:-1, 1, 0]
        u[1:-1, 0, 1] = - u[1:-1, 1, 1]
        # bot - no slip
        u[1:-1, -1, :] = - u[1:-1, -2, :]
        # left - no slip
        u[0, 1:-1, :] = - u[1, 1:-1, :]
        # right - no slip
        u[-1, 1:-1, :] = - u[-2, 1:-1, :]
        # corners
        u[ 0,  0, :] = (u[ 1,  0, :] + u[ 0,  1, :]) / 2
        u[-1,  0, :] = (u[-2,  0, :] + u[-1,  1, :]) / 2
        u[ 0, -1, :] = (u[ 1, -1, :] + u[ 0, -2, :]) / 2
        u[-1, -1, :] = (u[-1, -2, :] + u[-2, -1, :]) / 2
        
    
    def launch(self):
        
        self.curr_u = self.initial_u
        for step in (pbar := tqdm(range(1, self.steps))):
            self.setup_bc(self.curr_u)
            conv = self.compute_convection(self.curr_u)
            visc = self.compute_viscous_drag(self.curr_u)
            u_star = self.curr_u + self.dt * (-conv + visc)
            self.setup_bc(u_star)
            
            self.p = self.compute_updated_pressure(u_star)
            projected_u_star = self.project_velocity(u_star)
            #print(projected_u_star.mean())
            self.curr_u = projected_u_star
            if step % self.description_refresh == 0:
                div = self.compute_divergence(self.curr_u)
                pbar.set_description(f"[{step}] Divergence | min {Decimal(div.min()):.2E} | mean {Decimal(div.mean()):.2E} | max {Decimal(div.max()):.2E}")
            
            if step % self.dsave == 0:
                self.save_curr()
            # print(f"[Step {step}] u mean: {all_u[step].mean():.2f}, conv: {conv.mean():.2f}, visc: {visc.mean():.2f}, u_up mean: {u_update.mean():.2f},  inter u mean: {inter_u.mean():.2f} new pressure mean: {self.p.mean():.2f}")
        return CFDResult(self.saved_u, self)

    # based on file names only
    def run_or_load(self, save=True):
        np.seterr(over='raise')
        folder = Path('runs/')
        required = {f'{self.name}.json', f'{self.name}.npy'}
        
        exists = required.issubset({p.name for p in folder.iterdir() if p.is_file()})
        if not exists:
            print(f'Running simulation [{self.name}]')
            try:
                result = self.launch()
            except RuntimeWarning as e:
                print(e)
            if save:
                result.save()
            return result
        print(f'Loading simulation [{self.name}]')
        return CFDResult.load(self)
        
            
                 

In [ ]:
re = 400
simulations = [
    CFDSimulation(re, 10000, 1e-4, 100, 32),
    CFDSimulation(re, 100000, 1e-7, 100, 32),
]

results = [simu.run_or_load() for simu in simulations]

In [ ]:
for r in results[:-1]:
    r.energy_over_time(False)
results[-1].energy_over_time()

In [ ]:
snap = 90
for r in results[:-1]:
    r.profile_at_time(snap, False, False)
results[-1].profile_at_time(snap)

In [ ]:
results[-1].plot_velocities()